In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import timeit

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('/data/home/cou/yuli5/Audio/online_data_adaption/Mel_Audio_folder_digits',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
Perfomance = pd.read_csv('Performance_0915_Digit_Inputs.csv')

In [10]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [11]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = InceptionResNetV2(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    start = timeit.default_timer()
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:19]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    stop = timeit.default_timer()
    run_time = stop - start
    
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    participant_class = np.array(list(map(int, participant_class)))
    acc_s = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    Perfomance = Perfomance.append({'Feature_Extract': "INCEPTION_Resnet_v2",'Inputs': "MEL",'Time':run_time,
                                    'Partcp_Acc':acc_s,'Command_Acc':acc_c}, ignore_index=True)
    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0915_vgg.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

219062272/219055592 [==============================] - 5s 0us/step
Epoch 1/10
16/16 [==============================] - 7s 437ms/step - loss: 3.2975 - participant_output_loss: 1.4914 - command_output_loss: 1.8034 - command_output_1_loss: 0.0016 - participant_output_1_loss: 0.0011 - participant_output_accuracy: 0.3390 - command_output_accuracy: 0.5680 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2150 - val_loss: 2.5411 - val_participant_output_loss: 1.2527 - val_command_output_loss: 1.2877 - val_command_output_1_loss: 2.5440e-04 - val_participant_output_1_loss: 4.1047e-04 - val_participant_output_accuracy: 0.5280 - val_command_output_accuracy: 0.8440 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2680
Epoch 2/10
16/16 [==============================] - 3s 203ms/step - loss: 2.0580 - participant_output_loss: 1.0449 - command_output_loss: 1.0126 - command_output_1_loss: 2.4355e-04 - participant_output_1_loss: 2.4678e-04 - pa

16/16 [==============================] - 3s 214ms/step - loss: 0.1126 - participant_output_loss: 0.0652 - command_output_loss: 0.0471 - command_output_1_loss: 1.5774e-04 - participant_output_1_loss: 9.2127e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0910 - participant_output_1_accuracy: 0.2280 - val_loss: 0.2396 - val_participant_output_loss: 0.1650 - val_command_output_loss: 0.0743 - val_command_output_1_loss: 1.6880e-04 - val_participant_output_1_loss: 1.3185e-04 - val_participant_output_accuracy: 0.9720 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.1060 - val_participant_output_1_accuracy: 0.1360
Epoch 3/10
16/16 [==============================] - 3s 184ms/step - loss: 0.0954 - participant_output_loss: 0.0542 - command_output_loss: 0.0410 - command_output_1_loss: 1.4977e-04 - participant_output_1_loss: 8.2658e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_

Epoch 4/10
16/16 [==============================] - 4s 242ms/step - loss: 0.9542 - participant_output_loss: 0.5079 - command_output_loss: 0.4459 - command_output_1_loss: 1.5125e-04 - participant_output_1_loss: 2.1069e-04 - participant_output_accuracy: 0.9280 - command_output_accuracy: 0.9880 - command_output_1_accuracy: 0.0140 - participant_output_1_accuracy: 0.1490 - val_loss: 0.8865 - val_participant_output_loss: 0.5018 - val_command_output_loss: 0.3843 - val_command_output_1_loss: 1.3916e-04 - val_participant_output_1_loss: 2.4525e-04 - val_participant_output_accuracy: 0.9060 - val_command_output_accuracy: 0.9840 - val_command_output_1_accuracy: 0.0520 - val_participant_output_1_accuracy: 0.3020
Epoch 5/10
16/16 [==============================] - 4s 228ms/step - loss: 0.6606 - participant_output_loss: 0.3717 - command_output_loss: 0.2886 - command_output_1_loss: 1.4772e-04 - participant_output_1_loss: 2.0893e-04 - participant_output_accuracy: 0.9620 - command_output_accuracy: 0.9950

Epoch 6/10
16/16 [==============================] - 3s 209ms/step - loss: 0.0606 - participant_output_loss: 0.0322 - command_output_loss: 0.0282 - command_output_1_loss: 1.2405e-04 - participant_output_1_loss: 6.9587e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0860 - participant_output_1_accuracy: 0.1820 - val_loss: 0.1887 - val_participant_output_loss: 0.1352 - val_command_output_loss: 0.0533 - val_command_output_1_loss: 1.3623e-04 - val_participant_output_1_loss: 1.0476e-04 - val_participant_output_accuracy: 0.9720 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.0800 - val_participant_output_1_accuracy: 0.1720
Epoch 7/10
16/16 [==============================] - 3s 207ms/step - loss: 0.0537 - participant_output_loss: 0.0282 - command_output_loss: 0.0254 - command_output_1_loss: 1.1604e-04 - participant_output_1_loss: 6.3153e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 8/10
16/16 [==============================] - 3s 193ms/step - loss: 0.3069 - participant_output_loss: 0.1886 - command_output_loss: 0.1179 - command_output_1_loss: 1.7307e-04 - participant_output_1_loss: 2.1134e-04 - participant_output_accuracy: 0.9930 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1150 - participant_output_1_accuracy: 0.1650 - val_loss: 0.4237 - val_participant_output_loss: 0.2837 - val_command_output_loss: 0.1396 - val_command_output_1_loss: 1.9184e-04 - val_participant_output_1_loss: 2.2264e-04 - val_participant_output_accuracy: 0.9380 - val_command_output_accuracy: 0.9920 - val_command_output_1_accuracy: 0.1500 - val_participant_output_1_accuracy: 0.1220
Epoch 9/10
16/16 [==============================] - 3s 199ms/step - loss: 0.2397 - participant_output_loss: 0.1509 - command_output_loss: 0.0884 - command_output_1_loss: 1.7031e-04 - participant_output_1_loss: 1.8621e-04 - participant_output_accuracy: 0.9950 - command_output_accuracy: 1.0000

...

Epoch 10/10
16/16 [==============================] - 3s 178ms/step - loss: 0.1904 - participant_output_loss: 0.1219 - command_output_loss: 0.0681 - command_output_1_loss: 1.8745e-04 - participant_output_1_loss: 1.4764e-04 - participant_output_accuracy: 0.9960 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1460 - participant_output_1_accuracy: 0.2220 - val_loss: 0.3361 - val_participant_output_loss: 0.2361 - val_command_output_loss: 0.0997 - val_command_output_1_loss: 1.9688e-04 - val_participant_output_1_loss: 1.8579e-04 - val_participant_output_accuracy: 0.9500 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.1560 - val_participant_output_1_accuracy: 0.1440
Epoch 1/10
16/16 [==============================] - 4s 239ms/step - loss: 0.1589 - participant_output_loss: 0.1017 - command_output_loss: 0.0570 - command_output_1_loss: 1.7896e-04 - participant_output_1_loss: 1.3542e-04 - participant_output_accuracy: 0.9980 - command_output_accuracy: 1.000

Epoch 2/10
16/16 [==============================] - 3s 202ms/step - loss: 2.0121 - participant_output_loss: 0.9640 - command_output_loss: 1.0476 - command_output_1_loss: 3.0312e-04 - participant_output_1_loss: 2.8957e-04 - participant_output_accuracy: 0.7320 - command_output_accuracy: 0.9210 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2660 - val_loss: 1.6414 - val_participant_output_loss: 0.8153 - val_command_output_loss: 0.8255 - val_command_output_1_loss: 3.5302e-04 - val_participant_output_1_loss: 2.8005e-04 - val_participant_output_accuracy: 0.7880 - val_command_output_accuracy: 0.9600 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1080
Epoch 3/10
16/16 [==============================] - 3s 201ms/step - loss: 1.3138 - participant_output_loss: 0.6661 - command_output_loss: 0.6472 - command_output_1_loss: 3.4128e-04 - participant_output_1_loss: 2.1147e-04 - participant_output_accuracy: 0.8560 - command_output_accuracy

Epoch 4/10
16/16 [==============================] - 3s 183ms/step - loss: 0.0831 - participant_output_loss: 0.0467 - command_output_loss: 0.0362 - command_output_1_loss: 1.4597e-04 - participant_output_1_loss: 6.3684e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1120 - participant_output_1_accuracy: 0.2330 - val_loss: 0.2203 - val_participant_output_loss: 0.1558 - val_command_output_loss: 0.0642 - val_command_output_1_loss: 1.4903e-04 - val_participant_output_1_loss: 7.8564e-05 - val_participant_output_accuracy: 0.9680 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.1120 - val_participant_output_1_accuracy: 0.2040
Epoch 5/10
16/16 [==============================] - 3s 204ms/step - loss: 0.0722 - participant_output_loss: 0.0398 - command_output_loss: 0.0322 - command_output_1_loss: 1.3619e-04 - participant_output_1_loss: 5.7694e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 6/10
16/16 [==============================] - 3s 211ms/step - loss: 0.5528 - participant_output_loss: 0.2939 - command_output_loss: 0.2584 - command_output_1_loss: 2.5223e-04 - participant_output_1_loss: 2.3499e-04 - participant_output_accuracy: 0.9720 - command_output_accuracy: 0.9960 - command_output_1_accuracy: 0.1880 - participant_output_1_accuracy: 0.1860 - val_loss: 0.6048 - val_participant_output_loss: 0.3535 - val_command_output_loss: 0.2509 - val_command_output_1_loss: 2.3135e-04 - val_participant_output_1_loss: 2.4500e-04 - val_participant_output_accuracy: 0.9380 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.1600 - val_participant_output_1_accuracy: 0.1500
Epoch 7/10
16/16 [==============================] - 3s 215ms/step - loss: 0.3944 - participant_output_loss: 0.2187 - command_output_loss: 0.1753 - command_output_1_loss: 1.8575e-04 - participant_output_1_loss: 2.2949e-04 - participant_output_accuracy: 0.9850 - command_output_accuracy: 0.9990

Epoch 8/10
16/16 [==============================] - 3s 181ms/step - loss: 0.0551 - participant_output_loss: 0.0303 - command_output_loss: 0.0246 - command_output_1_loss: 1.0569e-04 - participant_output_1_loss: 6.0820e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1050 - participant_output_1_accuracy: 0.1790 - val_loss: 0.1822 - val_participant_output_loss: 0.1332 - val_command_output_loss: 0.0488 - val_command_output_1_loss: 1.2026e-04 - val_participant_output_1_loss: 8.9133e-05 - val_participant_output_accuracy: 0.9740 - val_command_output_accuracy: 0.9980 - val_command_output_1_accuracy: 0.0860 - val_participant_output_1_accuracy: 0.2440
Epoch 9/10
16/16 [==============================] - 3s 174ms/step - loss: 0.0497 - participant_output_loss: 0.0272 - command_output_loss: 0.0224 - command_output_1_loss: 9.7827e-05 - participant_output_1_loss: 5.5427e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

In [12]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


In [13]:
Perfomance.to_csv('Performance_0915_Digit_Inputs.csv',index=False)